In [2]:
#importing libraries
import pandas as pd
import seaborn as sns
from IPython.display import display

# Reading Data


In [3]:
data=pd.read_csv("Numeric data HR.csv")

In [4]:
display(data.info())
display(data.describe())
print("NAN")
nan_counts = data.isna().sum()
print(nan_counts[nan_counts > 0])

print("\nNULL")
null_counts = data.isnull().sum()
print(null_counts[null_counts > 0])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

None

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470.000000,1470.000000,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,...,1470.000000,1470.0,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
mean,36.923810,802.485714,9.192517,2.912925,1.0,1024.865306,2.721769,65.891156,2.729932,2.063946,...,2.712245,80.0,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,403.509100,8.106864,1.024165,0.0,602.024335,1.093082,20.329428,0.711561,1.106940,...,1.081209,0.0,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,102.000000,1.000000,1.000000,1.0,1.000000,1.000000,30.000000,1.000000,1.000000,...,1.000000,80.0,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,465.000000,2.000000,2.000000,1.0,491.250000,2.000000,48.000000,2.000000,1.000000,...,2.000000,80.0,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,802.000000,7.000000,3.000000,1.0,1020.500000,3.000000,66.000000,3.000000,2.000000,...,3.000000,80.0,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,1157.000000,14.000000,4.000000,1.0,1555.750000,4.000000,83.750000,3.000000,3.000000,...,4.000000,80.0,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000
max,60.000000,1499.000000,29.000000,5.000000,1.0,2068.000000,4.000000,100.000000,4.000000,5.000000,...,4.000000,80.0,3.000000,40.000000,6.000000,4.000000,40.000000,18.000000,15.000000,17.000000


NAN
Series([], dtype: int64)

NULL
Series([], dtype: int64)


# Data preprocessing

Removing unnecessary columns


In [5]:
columns_to_drop = ['EmployeeCount', 'StandardHours', 'Over18']  # These columns have no variance
data = data.drop(columns=columns_to_drop)

Outliars

In [6]:
def remove_outliers(data):
    df_clean = data.copy()
    initial_rows = len(df_clean)
    
    for col in df_clean.select_dtypes(exclude=['object']).columns:
        Q1 = df_clean[col].quantile(0.15)
        Q3 = df_clean[col].quantile(0.85)
        IQR = Q3 - Q1
        df_clean = df_clean[(df_clean[col] >= Q1 - 1.5 * IQR) & 
                           (df_clean[col] <= Q3 + 1.5 * IQR)]
    
    removed = initial_rows - len(df_clean)
    display(f"Removed {removed} outliers ({removed/initial_rows:.2%} of data)")
    return df_clean
data=remove_outliers(data)

'Removed 51 outliers (3.47% of data)'

Feature Engineering

In [7]:
# Create any new features
data['Income_per_Year'] = data['MonthlyIncome'] * 12
data['Years_Since_First_Job'] = data['TotalWorkingYears'] - data['YearsAtCompany']

Feature Sepration

In [8]:
numeric_features = data.select_dtypes(exclude=['object']).columns
categorical_features = data.select_dtypes(include=['object']).columns

print("\nNumeric Features:", list(numeric_features))
print("Categorical Features:", list(categorical_features))


Numeric Features: ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'Income_per_Year', 'Years_Since_First_Job']
Categorical Features: ['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender', 'JobRole', 'MaritalStatus', 'OverTime']


Encoding

In [9]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [10]:
numeric_data=data.select_dtypes(exclude=['object'])
categorical_feature=data.select_dtypes(include=['object'])

encoder=LabelEncoder()
for col in categorical_feature.columns:
    data[col]=encoder.fit_transform(categorical_feature[col])

Scalling

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
data[numeric_features] = scaler.fit_transform(data[numeric_features])

In [12]:
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Income_per_Year,Years_Since_First_Job
0,0.502741,1,2,0.735542,2,-1.013435,-0.885708,1,-1.699990,-0.659145,...,-0.927061,-0.374620,-2.170721,-2.505455,-0.065650,-0.007362,-0.706936,0.313286,-0.046003,-0.376082
1,1.397134,0,1,-1.303654,1,-0.146161,-1.863778,1,-1.698333,0.255153,...,0.239162,-0.093655,0.156340,0.341015,0.751525,0.863249,-0.332769,0.901320,-0.240924,-0.696791
2,0.055545,1,2,1.407015,1,-0.889539,-0.885708,4,-1.695019,1.169451,...,-0.927061,-0.515103,0.156340,0.341015,-1.291413,-1.168177,-0.706936,-1.156797,-0.927549,0.425691
3,-0.391651,0,1,1.454092,1,-0.765643,1.070432,1,-1.693362,1.169451,...,-0.927061,-0.374620,0.156340,0.341015,0.342937,0.863249,0.415566,-1.156797,-0.742567,-0.696791
4,-1.062446,0,2,-0.530593,1,-0.889539,-1.863778,3,-1.690047,-1.573443,...,0.239162,-0.655586,0.156340,0.341015,-0.882826,-0.587770,0.041398,-0.568764,-0.616309,-0.055373


Memory optimisation

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1419 entries, 0 to 1469
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1419 non-null   float64
 1   Attrition                 1419 non-null   int64  
 2   BusinessTravel            1419 non-null   int64  
 3   DailyRate                 1419 non-null   float64
 4   Department                1419 non-null   int64  
 5   DistanceFromHome          1419 non-null   float64
 6   Education                 1419 non-null   float64
 7   EducationField            1419 non-null   int64  
 8   EmployeeNumber            1419 non-null   float64
 9   EnvironmentSatisfaction   1419 non-null   float64
 10  Gender                    1419 non-null   int64  
 11  HourlyRate                1419 non-null   float64
 12  JobInvolvement            1419 non-null   float64
 13  JobLevel                  1419 non-null   float64
 14  JobRole      

In [14]:
import numpy as np

for col in data.select_dtypes(include=['int64']).columns:
    data[col]=data[col].astype(np.int8)
for col in data.select_dtypes(include=['float64']).columns:
    data[col]=data[col].astype(np.float32)

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1419 entries, 0 to 1469
Data columns (total 34 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       1419 non-null   float32
 1   Attrition                 1419 non-null   int8   
 2   BusinessTravel            1419 non-null   int8   
 3   DailyRate                 1419 non-null   float32
 4   Department                1419 non-null   int8   
 5   DistanceFromHome          1419 non-null   float32
 6   Education                 1419 non-null   float32
 7   EducationField            1419 non-null   int8   
 8   EmployeeNumber            1419 non-null   float32
 9   EnvironmentSatisfaction   1419 non-null   float32
 10  Gender                    1419 non-null   int8   
 11  HourlyRate                1419 non-null   float32
 12  JobInvolvement            1419 non-null   float32
 13  JobLevel                  1419 non-null   float32
 14  JobRole      

In [16]:
data.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,...,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Income_per_Year,Years_Since_First_Job
0,0.502741,1,2,0.735542,2,-1.013435,-0.885708,1,-1.699990,-0.659145,...,-0.927061,-0.374620,-2.170721,-2.505455,-0.065650,-0.007362,-0.706936,0.313286,-0.046003,-0.376082
1,1.397134,0,1,-1.303654,1,-0.146161,-1.863778,1,-1.698333,0.255153,...,0.239162,-0.093655,0.156340,0.341015,0.751525,0.863249,-0.332769,0.901320,-0.240924,-0.696791
2,0.055545,1,2,1.407015,1,-0.889539,-0.885708,4,-1.695019,1.169451,...,-0.927061,-0.515103,0.156340,0.341015,-1.291413,-1.168177,-0.706936,-1.156797,-0.927549,0.425691
3,-0.391651,0,1,1.454092,1,-0.765643,1.070432,1,-1.693362,1.169451,...,-0.927061,-0.374620,0.156340,0.341015,0.342937,0.863249,0.415566,-1.156797,-0.742567,-0.696791
4,-1.062446,0,2,-0.530593,1,-0.889539,-1.863778,3,-1.690047,-1.573443,...,0.239162,-0.655586,0.156340,0.341015,-0.882826,-0.587770,0.041398,-0.568764,-0.616309,-0.055373
